# Part 4: generate 20000-movie poster data to prepare for deep learning

In [3]:
import json
import urllib
import cStringIO
from PIL import Image
from imdb import IMDb
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import time
import ast

In [3]:
# 20，000 dataset creation

random1 = urllib.urlopen("https://api.themoviedb.org/3/discover/movie?api_key=2dc6c9f1d17bd39dcbaef83321e1b5a3&language=en-US&page=1")
random1_json = json.loads(random1.read())
random_movie_data_json = random1_json["results"]

pages = range(2,1001)
# np.random.shuffle(pages)
# sampled_pages = pages[:500]


# need to sleep in order to not return an error: limitation 40 requests per 10s
for i in range(len(pages)):
    if i%39 == 0:
        time.sleep(7)

    tmp_url = "https://api.themoviedb.org/3/discover/movie?api_key=2dc6c9f1d17bd39dcbaef83321e1b5a3&language=en-US&page=" + str(pages[i])
    tmp_page = urllib.urlopen(tmp_url)
    tmp_json = json.loads(tmp_page.read())
    for movie in tmp_json["results"]:
        random_movie_data_json.append(movie)
        
    if i% 100 == 0:
        print i


0
100
200
300
400
500
600
700
800
900


In [4]:
genre_ids, overview, popularity, poster_path, title, vote_average, vote_count, release_date, movie_id = [], [], [], [], [], [], [], [], []
for movie in random_movie_data_json:
    genre_ids.append(movie["genre_ids"])
    overview.append(movie["overview"])
    popularity.append(movie["popularity"])
    poster_path.append(movie["poster_path"])
    title.append(movie["title"])
    vote_average.append(movie["vote_average"])
    vote_count.append(movie["vote_count"])
    release_date.append(movie["release_date"])
    movie_id.append(movie["id"])

data = {'title': title, 'overview': overview, 'popularity': popularity, 'release_date': release_date, 'poster_path': poster_path, 'vote_average': vote_average,  'vote_count': vote_count, 'genre_ids': genre_ids, 'movie_id': movie_id}
ran_df = pd.DataFrame(data = data)

In [7]:
# get genre list
genre_list = urllib.urlopen("https://api.themoviedb.org/3/genre/movie/list?api_key=2dc6c9f1d17bd39dcbaef83321e1b5a3&language=en-US")

genre_list_json = json.loads(genre_list.read()) 

genre_lst = {}
for i in genre_list_json['genres']:
    genre_lst[i['id']] = str(i['name'])

In [12]:
genre_lst.keys()

[10752,
 80,
 10402,
 35,
 36,
 37,
 53,
 9648,
 12,
 10770,
 14,
 16,
 18,
 99,
 878,
 27,
 28,
 10749,
 10751]

In [29]:
a = str(movie_20000_df.release_date[0])
f= a.split('-')
b = datetime.date(int(f[0]), int(f[1]), int(f[2]))
to_integer(b)

20170316

In [31]:
# process new features
movie_20000_df = ran_df.dropna()

labels = []
for i in movie_20000_df.genre_ids:
    label_matrix = np.zeros(len(genre_lst.keys()), dtype=int)
    for j in i:
        if j in genre_lst.keys():
            label_matrix[genre_lst.keys().index(j)] = 1
    labels.append(label_matrix)
movie_20000_df['labels'] = labels

# convert dates
import datetime
def to_integer(dt_time):
    return 10000*dt_time.year + 100*dt_time.month + dt_time.day

int_dates =[]

for i in movie_20000_df.release_date:
    f = str(i).split('-')
    try:
        ff = (int(f[0]), int(f[1]), int(f[2]))
    except:
        print i
    a = datetime.date(ff[0], ff[1], ff[2])
    int_dates.append(to_integer(a))

movie_20000_df['int_dates'] = int_dates

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
# to locate buffer overflow error
import csv
with open(r"20000_movie_meta2.csv", 'rb') as f:
    reader = csv.reader(f)
    linenumber = 1
    try:
        for row in reader:
            linenumber += 1
    except Exception as e:
        print (("Error line %d: %s %s" % (linenumber, str(type(e)), e.message)))
# movie_20000_df.to_csv('20000_movie_meta.csv', encoding = 'utf-8')

Error line 1: <class '_csv.Error'> line contains NULL byte


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6


In [70]:
# after correcting, we have a good csv file
movie_20000_df = pd.read_csv('20000_movie_meta_good.csv')

In [71]:
# now we download all the posters and put them into a df
# this process takes 2.5 hours
imgs = []
for i in range(len(movie_20000_df.poster_path[:10])):
    if i%39 == 0:
        # sleep
        time.sleep(7)
    try:
        url = "https://image.tmdb.org/t/p/w500" + movie_20000_df.poster_path[i]
    except:
        print "error"
        url = "https://image.tmdb.org/t/p/w500"+ '/ylXCdC106IKiarftHkcacasaAcb.jpg'
    tmp_poster = cStringIO.StringIO(urllib.urlopen(url).read())
    img = Image.open(tmp_poster) 
    imgs.append(img)
#     if i %100 == 0:
#         print i

In [72]:
# take a peek
imgs[:10]

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x750 at 0x2C1277C90>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x750 at 0x2C1277A10>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x750 at 0x2C1277BD0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x750 at 0x2C1277E10>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x750 at 0x2C1277E90>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x750 at 0x2C1277910>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x750 at 0x2C1277FD0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x750 at 0x2C1277C50>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x750 at 0x2C1277DD0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x750 at 0x2C1277C10>]

In [73]:
# create rgb arrays for these images
# 0 padding to those with less than 500 width
# so now all rgb pixels will have the same format
RGB = []
for img in imgs:
    tmp = img.load()
    pixels = []
    # crop if larger than 750
    if (img.size[1] > 750):
        for i in range(img.size[0]):
            for j in range(750):
                pixels.append(tmp[i,j])
    else: 
        for i in range(img.size[0]):
            for j in range(img.size[1]):
                pixels.append(tmp[i,j])
    # add 0 paddings if less than 750
    if (img.size[1] < 750):
        for p in range(img.size[0]):
            for q in range(img.size[1], 750):
                pixels.append((0,0,0))
    RGB.append(pixels)    

In [76]:
data_img = {'movie_id': movie_20000_df.movie_id,'genre_ids': movie_20000_df.genre_ids, 'RGB': RGB, 'imgs':imgs}
img_df = pd.DataFrame(data = data_img)
img_df.head()

,RGB,genre_ids,imgs,movie_id
0,"[(12, 32, 65), (11, 31, 64), (21, 41, 74), (28...","[14, 10402, 10749]",<PIL.JpegImagePlugin.JpegImageFile image mode=...,321612
1,"[(7, 9, 8), (5, 7, 6), (7, 9, 8), (7, 9, 8), (...","[28, 18, 878]",<PIL.JpegImagePlugin.JpegImageFile image mode=...,263115
2,"[(92, 79, 107), (89, 76, 104), (95, 80, 109), ...","[16, 35, 18, 10751, 10402]",<PIL.JpegImagePlugin.JpegImageFile image mode=...,335797
3,"[(140, 51, 17), (135, 46, 12), (133, 44, 10), ...","[28, 12, 14]",<PIL.JpegImagePlugin.JpegImageFile image mode=...,293167
4,"[(255, 255, 255), (255, 255, 255), (255, 255, ...","[28, 80, 53]",<PIL.JpegImagePlugin.JpegImageFile image mode=...,337339


In [77]:
# produce a csv
img_df.to_csv('imgs.csv')